# Molecular Energies

Let's see how we can use these abstract gates to model something from the real world like molecular energies. In quantum chemistry, the energy levels of electrons in a molecule can be represented using quantum states, and we can use quantum gates to manipulate these states.

## 1. The Hamiltonian

But first what is the energy of a molecule? In quantum mechanics, the energy of a system is described by an operator called the Hamiltonian.
It is basically a sum of terms that represent different interactions within the molecule.

The Hamiltonian can be expressed mathematically as:

$$\hat{H}_{tot} = \hat{T}_N + \hat{T}_e + \hat{V}_{eN} + \hat{V}_{NN} + \hat{V}_{ee}$$

Where:
- $\hat{T}_N$ is the kinetic energy of the nuclei (protons)
- $\hat{T}_e$ is the kinetic energy of the electrons
- $\hat{V}_{eN}$ is the potential energy due to attraction between electrons and nuclei
- $\hat{V}_{NN}$ is the potential energy due to repulsion between nuclei
- $\hat{V}_{ee}$ is the potential energy due to repulsion between electrons

Usually, we simplify this Hamiltonian using the Born-Oppenheimer approximation, which assumes that the nuclei are stationary compared to the fast-moving electrons. This allows us to focus on the electronic part of the Hamiltonian:

$$\hat{H}_{elec} = \hat{T}_e + \hat{V}_{eN} + \hat{V}_{ee}$$

### Hydrogen Molecule

To have something to work with let's consider the simplest molecule, the hydrogen molecule (H₂). It consists of two protons and two electrons. We will use pennylane to get the Hamiltonian for this molecule.

In [1]:
# Building Hamiltonian for H₂ using pennylane
import pennylane as qml
from pennylane import numpy as pnp

# Define the molecular parameters
symbols = ['H', 'H']
bond_length = 0.74  # in Angstroms
geometry = pnp.array([[0.0, 0.0, 0.0],
                      [0.0, 0.0, bond_length]], dtype=float)

# Build molecule the qubit Hamiltonian
h2_mol = qml.qchem.Molecule(
    symbols=symbols,
    coordinates=geometry,
    charge=0,
    mult=1,
    basis_name='sto-3g',
)
H, n_qubits = qml.qchem.molecular_hamiltonian(
    molecule=h2_mol,
    mapping='jordan_wigner',
)
hf_state = qml.qchem.hf_state(
    electrons=2, orbitals=n_qubits
)

print("Number of qubits:", n_qubits)
print("Hamiltonian:")
print(H)
print("Hartree-Fock state:", hf_state)

Number of qubits: 4
Hamiltonian:
0.7784107516237684 * I([0, 1, 2, 3]) + 0.23718740030498442 * Z(0) + 0.23718740030498436 * Z(1) + 0.18456105296876102 * (Z(0) @ Z(1)) + -0.4612495707632222 * Z(2) + 0.14065700585010016 * (Z(0) @ Z(2)) + 0.18170118517545314 * (Z(1) @ Z(2)) + 0.04104417932535298 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.04104417932535298 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.04104417932535298 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 0.04104417932535298 * (X(0) @ Y(1) @ Y(2) @ X(3)) + -0.46124957076322226 * Z(3) + 0.18170118517545314 * (Z(0) @ Z(3)) + 0.14065700585010016 * (Z(1) @ Z(3)) + 0.1917875056241911 * (Z(2) @ Z(3))
Hartree-Fock state: [1 1 0 0]


We can see that the Hamiltonian will need four qubits to represent the two electrons in two different orbitals. If we denote the qubits as follows:

- Qubit 0: Spin up electron in orbital 1
- Qubit 1: Spin down electron in orbital 1
- Qubit 2: Spin up electron in orbital 2
- Qubit 3: Spin down electron in orbital 2

Of course, there are many ways to represent where the electrons are, but this is one common way called the Jordan-Wigner mapping.
We will discuss this mapping in more detail later.
For now, we will use the hartree-fock state as our initial state, meaning that we will put the two electrons in the lowest energy orbital.

Note that there is this rule called the Pauli exclusion principle. 
It states that no two electrons can occupy the same quantum state simultaneously. 
This means that we cannot have more than two electrons in the same orbital, and if they are in the same orbital, they must have opposite spins. 
Thus, we will always just need two qubits per orbital.

Now, to the most interesting part, the Hamiltonian for the hydrogen molecule.
The Hamiltonian is a observable, meaning that it can be represented as a sum of Pauli operators.
The specific Hamiltonian we see here is sort of a recipe for how to calculate the energy of the hydrogen molecule using quantum gates:

- The first term `0.7784 * I([0, 1, 2, 3])` represents a constant energy shift. The `I` operator is the identity operator, which means it doesn't change the state of the qubits. This term just adds a constant value to the energy.
- The second term `0.2371 * Z(0)` means that we measure qubit 0 in the Z-basis. When measuring state $|0\rangle$, we add $0.2371 \cdot 1$ to the energy, and when measuring state $|1\rangle$, we add $0.2371 \cdot (-1)$ to the energy.
- The third term is the same as the second term but for qubit 1.
- As for the fourth term `0.1845 * (Z(0) @ Z(1))`, it means that we measure both qubits 0 and 1 in the Z-basis and multiply the results. If both qubits are in state $|0\rangle$, we add $0.1845 \cdot 1 \cdot 1$ to the energy. If qubit 0 is in state $|0\rangle$ and qubit 1 is in state $|1\rangle$, we add $0.1845 \cdot 1 \cdot (-1)$ to the energy, and so on for the other combinations.
- When we see terms like `0.0410 * (Y(0) @ X(1) @ X(2) @ Y(3))`, it means that we measure qubit 0 in the Y-basis, qubit 1 in the X-basis, qubit 2 in the X-basis, and qubit 3 in the Y-basis, and then multiply the results together. Of course, we then multiply the result by coefficient to get the contribution to the energy.

In [2]:
# Let's see how much energy the hydrogen molecule has with our initial state
dev = qml.device('default.qubit', wires=n_qubits)
@qml.qnode(dev)
def energy_hf():
    qml.BasisState(hf_state, wires=range(n_qubits))
    return qml.expval(H)
print("Energy of the Hartree-Fock state:", energy_hf())

Energy of the Hartree-Fock state: -0.8868310139707993


The energy we get is about -0.8868 Hartree, which is a unit of energy commonly used in quantum chemistry (1 Hartree is approximately 27.2114 eV or 4.3597e-18 Joules).

## 2. From math to creation/annihilation operators

Let's see how we actually get this Hamiltonian from the mathematical description of the hydrogen molecule, instead of just relying on pennylane to give it to us.

The non-relativistic electronic Hamiltonian in second quantization is given by:

$$\hat{H} = E_{nuc} + \sum_{pq} h_{pq} a_p^\dagger a_q + \frac{1}{2} \sum_{pqrs} h_{pqrs} a_p^\dagger a_q^\dagger a_r a_s$$

Where:
- $E_{nuc}$ is the nuclear repulsion energy (constant because we assume the nuclei are stationary)
- $h_{pq}$ are the one-electron integrals (kinetic & electron-nuclear attraction). What it does is to sum over all pairs of orbitals (p, q) and calculate the contribution to the energy from an electron moving from orbital q to orbital p. $a_p^\dagger$ creates an electron in orbital p, and $a_q$ annihilates (removes) an electron from orbital q.
- $h_{pqrs}$ are the two-electron integrals (electron-electron repulsion). This term sums over all combinations of four orbitals (p, q, r, s) and calculates the contribution to the energy from two electrons interacting with each other while moving between these orbitals. $a_p^\dagger$ and $a_q^\dagger$ create electrons in orbitals p and q, while $a_r$ and $a_s$ annihilate electrons in orbitals r and s. The factor of $\frac{1}{2}$ is included to avoid double-counting the interactions between pairs of electrons.

So in summary, this Hamiltonian describes the total energy of a system of electrons in a molecule by considering their kinetic energy, their attraction to the nuclei, and their repulsion from each other.

Pennylane uses classical computational chemistry software to calculate the integrals $h_{pq}$ and $h_{pqrs}$, and then constructs the Hamiltonian using these integrals. Then it uses a mapping (like Jordan-Wigner) to convert the creation and annihilation operators into qubit operators (Pauli matrices). This will be the topic of the next section.

In [3]:
# Compute the core constant and the one- and two-electron integrals
# Running on a different h2 object to avoid changing or
symbols  = ['H', 'H']
geometry = pnp.array([[0.0, 0.0, 0.0], [0.0, 0.0, 0.74]], requires_grad = False)
alpha = pnp.array([[3.42525091, 0.62391373, 0.1688554],
                  [3.42525091, 0.62391373, 0.1688554]], requires_grad=True)
mol = qml.qchem.Molecule(symbols, geometry, alpha=alpha, unit='Angstrom')
args = [alpha]

E_core, h1, h2 = qml.qchem.electron_integrals(mol)(*args)

print("Core constant (E_nuc):")
print(f"  {E_core[0]: .12f}")

print("\nOne-electron integrals h[p,q]:")
for p in range(h1.shape[0]):
    for q in range(h1.shape[1]):
        val = h1[p, q]
        if abs(val) < 1e-12:
            val = 0.0
        print(f"  h[{p},{q}] = {val: .12f}")

print("\nTwo-electron integrals h[p,q,r,s] (omitting ~0 terms):")
for p in range(h2.shape[0]):
    for q in range(h2.shape[1]):
        for r in range(h2.shape[2]):
            for s in range(h2.shape[3]):
                val = h2[p, q, r, s]
                if abs(val) < 1e-12:
                    continue
                print(f"  h[{p},{q},{r},{s}] = {val: .12f}")

Core constant (E_nuc):
   0.715104339058

One-electron integrals h[p,q]:
  h[0,0] = -1.253309783263
  h[0,1] =  0.000000000000
  h[1,0] =  0.000000000000
  h[1,1] = -0.475068847654

Two-electron integrals h[p,q,r,s] (omitting ~0 terms):
  h[0,0,0,0] =  0.674755922309
  h[0,0,1,1] =  0.181210458478
  h[0,1,0,1] =  0.181210458478
  h[0,1,1,0] =  0.663711389773
  h[1,0,0,1] =  0.663711389773
  h[1,0,1,0] =  0.181210458478
  h[1,1,0,0] =  0.181210458478
  h[1,1,1,1] =  0.697651485023


### Quick analysis of the core constant and the one-/two-electron integrals

#### Core constant (nuclear repulsion)
- $E_{\mathrm{nuc}} = 0.715104339058$  
  Positive nucleus–nucleus Coulomb repulsion; constant energy shift.

#### One-electron integrals $h_{pq}$
Kinetic + electron–nucleus attraction contributions (diagonal in this basis):
- $h_{00} = -1.253309783263$
- $h_{01} = h_{10} = 0.000000000000$
- $h_{11} = -0.475068847654$

Interpretation:
- Diagonals are orbital self-energies.
- Off-diagonals vanish → no single-electron coupling between 0 and 1 in this canonical (orthogonal) orbital set.

#### Two-electron integrals $h_{pqrs}$ (electron–electron repulsion)
Non-negligible terms:
- $h_{0000} = 0.674755922309$
- $h_{0011} = h_{1100} = 0.181210458478$
- $h_{0101} = h_{1010} = 0.181210458478$
- $h_{0110} = h_{1001} = 0.663711389773$
- $h_{1111} = 0.697651485023$

Interpretation:
- $h_{0000}, h_{1111}$: same-orbital (opposite-spin) Coulomb repulsion.
- $h_{0101}$ etc.: direct (Coulomb) interaction between densities of orbitals 0 and 1 ($J_{01}$).
- $h_{0110}$ etc.: exchange interaction ($K_{01}$), distinct from $J_{01}$ and typically smaller or comparable; here noticeably larger than $J_{01}$ due to minimal basis characteristics.

Symmetry: Standard permutational relations reduce the number of unique values.

# 3. Fermion to qubit mapping